# Graph DB Neo4j chain
지식 그래프 데이터베이스로 많이 사용되는 Neo4j와 LLM을 함께 사용하는 방법에 대해 알아봅니다

## Python Package
예시에서 사용할 몇 가지 주요 패키지를 설치합니다. 아래 주석처리된 명령어를 통해 Jupyter notebook 환경에서 바로 설치하실 수 있습니다.

In [15]:
# !pip install langchain -U
# !pip install neo4j
# !pip install python-dotenv

## Neo4j 설치
Docker 환경에서 Neo4j 데이터베이스 서버를 활성화 합니다. 
아래 Docker 컨테이너 실행 명령어를 통해 Neo4j를 설치하고 서버를 활성화 합니다. 
```
docker run \
--name neo4j \
-p 7474:7474 -p 7687:7687 \
-d \
-e NEO4J_AUTH=neo4j/pleaseletmein \
-e NEO4J_PLUGINS=\[\"apoc\"\]  \
-e NEO4J_apoc_export_file_enabled=true \
-e NEO4J_apoc_import_file_enabled=true \
-e NEO4J_apoc_import_file_use__neo4j__config=true \
neo4j:latest
```
위 명령어를 실행하면 Neo4j 서버가 실행되고, http://localhost:7474/ 에서 Neo4j Browser를 통해 Neo4j 서버에 접속할 수 있습니다.
참고로 Neo4j Browser의 기본 계정은 neo4j/neo4j 이지만 옵션으로 설정한 NEO4J_AUTH=neo4j/pleaseletmein 을 통해 neo4j/pleaseletmein 계정으로 접속할 수 있습니다.

## LangChain with Neo4j
- LangChain 패키지에서 지원하는 Neo4j 연결 작업을 수행합니다
- 예시 쿼리를 통해 신규 그래프 데이터를 추가하고 Neo4j 서버에 저장합니다
- LangChain으로 연결된 Neo4j 전용 Agent가 대화 과정에서 Neo4j를 참고하는지 테스트 합니다

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from dotenv import load_dotenv
# load the environment variables from the .env file in the root directory of the project
load_dotenv(verbose=True, override=True)
# delete the load_dotenv function from the namespace
del load_dotenv

In [78]:
graph = Neo4jGraph(
    url="bolt://localhost:7687", 
    username="neo4j", 
    password="pleaseletmein"
)

In [79]:
graph.query(
"""
MERGE (m:Movie {name:"Top Gun"})
WITH m
UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
MERGE (a:Actor {name:actor})
MERGE (a)-[:ACTED_IN]->(m)
"""
)

[]

In [87]:
graph.refresh_schema()

In [88]:
print(graph.get_schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [82]:
# Query the graph
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [83]:
chain.run("Who played in Top Gun?")



> Entering new  chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie {name: 'Top Gun'})
RETURN a.name
Full Context:
[{'a.name': 'Tom Cruise'}, {'a.name': 'Val Kilmer'}, {'a.name': 'Anthony Edwards'}, {'a.name': 'Meg Ryan'}]

> Finished chain.


'Tom Cruise, Val Kilmer, Anthony Edwards, and Meg Ryan played in Top Gun.'

In [86]:
chain.run("Clean the graph database.")



> Entering new  chain...
Generated Cypher:
MATCH (n)
DETACH DELETE n
Full Context:
[]

> Finished chain.


"I'm sorry, but without any specific information about the graph database or the cleaning process, I'm unable to provide a helpful answer. Could you please provide more details or clarify your question?"

## Synthesize Graph Data
- Theme: Food Ingredients and Recipes

In [30]:
import json
import ast
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [111]:
synth_prompt = "Imagin a random dish. Describe the ingredients used, the cooking process, the taste of each ingredient, and the overall flavor of the dish."
synth_llm = OpenAI(temperature=0.5, verbose=True)
synth_result = synth_llm.predict(synth_prompt)
print(synth_result)



This dish is a savory and flavorful mix of vegetables and proteins. The base of the dish is a mix of cooked quinoa and white rice, which provides a nutty and slightly sweet flavor. The vegetables used are diced red bell peppers, carrots, and onions, which are sautéed in olive oil until soft. The proteins used are diced chicken, which is cooked until golden brown and slightly crispy. The flavor of the chicken is slightly sweet and savory. The dish is seasoned with a blend of herbs and spices, including oregano, cumin, and garlic powder. This gives the dish a slightly smoky and earthy flavor. Finally, the dish is finished with a squeeze of fresh lemon juice, which adds a bright and acidic note to the dish. The overall flavor of the dish is savory and slightly sweet, with a hint of smokiness from the herbs and spices.


In [112]:
preprocess_instruction = f"List all the relational information that can be gleaned from the given text: {synth_result}"
preprocess_llm = OpenAI(temperature=0.5, verbose=True)
preprocess_result = preprocess_llm.predict(preprocess_instruction)
print(preprocess_result)



Relational information: 
- Base of the dish is a mix of cooked quinoa and white rice
- Vegetables used are diced red bell peppers, carrots, and onions
- Proteins used are diced chicken
- Seasoned with a blend of herbs and spices including oregano, cumin, and garlic powder
- Finished with a squeeze of fresh lemon juice


In [113]:
knowledge_graph_parsing_instruction = f"Divide and list all the information that can be gleaned from the given data into units suitable for storage in a graph database.: {preprocess_result}"
kg_llm = OpenAI(temperature=0.5, verbose=True)
kg_result = kg_llm.predict(knowledge_graph_parsing_instruction)
print(kg_result)



Nodes: 
- Base
- Vegetables
- Proteins
- Herbs and Spices
- Lemon Juice

Relationships: 
- Quinoa and Rice
- Red Bell Peppers and Carrots
- Onions and Chicken
- Oregano and Cumin
- Garlic Powder and Lemon Juice


In [120]:
kg_meta_node_labels = chain.run(f"Summarize the node labels")
kg_meta_edge_labels = chain.run(f"Summarize the relationship labels")



> Entering new  chain...
Generated Cypher:
MATCH (n)
RETURN DISTINCT labels(n) AS NodeLabels
Full Context:
[{'NodeLabels': ['Ingredient']}, {'NodeLabels': ['Action']}, {'NodeLabels': ['Recipe']}, {'NodeLabels': ['Preparation']}, {'NodeLabels': ['Seasoning']}, {'NodeLabels': ['FlavorProfile']}, {'NodeLabels': ['OverallFlavor']}, {'NodeLabels': ['Node']}, {'NodeLabels': ['Method']}, {'NodeLabels': ['Result']}]

> Finished chain.


> Entering new  chain...
Generated Cypher:
MATCH ()-[r]->()
RETURN DISTINCT type(r)
Full Context:
[{'type(r)': 'HAS_PREPARATION'}, {'type(r)': 'REQUIRES_SEASONING'}, {'type(r)': 'HAS_FLAVOR_PROFILE'}, {'type(r)': 'SERVED_WITH'}, {'type(r)': 'SEASONED_WITH'}, {'type(r)': 'ROASTED_WITH'}, {'type(r)': 'MADE_FROM'}, {'type(r)': 'SERVED_ON_TOP_OF'}, {'type(r)': 'TASTES_LIKE'}, {'type(r)': 'OVERALL_FLAVOR'}]

> Finished chain.
The node labels include Ingredient, Action, Recipe, Preparation, Seasoning, FlavorProfile, OverallFlavor, Node, Method, and Result.
The rela

In [121]:
print(kg_meta_node_labels)
print(kg_meta_edge_labels)

The node labels include Ingredient, Action, Recipe, Preparation, Seasoning, FlavorProfile, OverallFlavor, Node, Method, and Result.
The relationship labels provided include "HAS_PREPARATION", "REQUIRES_SEASONING", "HAS_FLAVOR_PROFILE", "SERVED_WITH", "SEASONED_WITH", "ROASTED_WITH", "MADE_FROM", "SERVED_ON_TOP_OF", "TASTES_LIKE", and "OVERALL_FLAVOR".


In [122]:
graph_data_update_prompt = f"""
Node labels: {kg_meta_node_labels}
Relationship labels: {kg_meta_edge_labels}
Instruction: Save the contents into the database
Contents: {kg_result}
"""
# chain.run(f"Save the contents into the database:\n{kg_result}")
chain.run(graph_data_update_prompt)



> Entering new  chain...
Generated Cypher:
CREATE (Base:Ingredient {name: "Base"})
CREATE (Vegetables:Ingredient {name: "Vegetables"})
CREATE (Proteins:Ingredient {name: "Proteins"})
CREATE (HerbsAndSpices:Ingredient {name: "Herbs and Spices"})
CREATE (LemonJuice:Ingredient {name: "Lemon Juice"})

CREATE (QuinoaAndRice:Action {name: "Quinoa and Rice"})
CREATE (RedBellPeppersAndCarrots:Action {name: "Red Bell Peppers and Carrots"})
CREATE (OnionsAndChicken:Action {name: "Onions and Chicken"})
CREATE (OreganoAndCumin:Action {name: "Oregano and Cumin"})
CREATE (GarlicPowderAndLemonJuice:Action {name: "Garlic Powder and Lemon Juice"})

CREATE (Recipe:Node {name: "Recipe"})
CREATE (Preparation:Node {name: "Preparation"})
CREATE (Seasoning:Node {name: "Seasoning"})
CREATE (FlavorProfile:Node {name: "FlavorProfile"})
CREATE (OverallFlavor:Node {name: "OverallFlavor"})
CREATE (Method:Node {name: "Method"})
CREATE (Result:Node {name: "Result"})

CREATE (Base)-[:MADE_FROM]->(QuinoaAndRice)
CRE

'I will save the following contents into the database:\n\nNodes: \n- Base\n- Vegetables\n- Proteins\n- Herbs and Spices\n- Lemon Juice\n\nRelationships: \n- Quinoa and Rice\n- Red Bell Peppers and Carrots\n- Onions and Chicken\n- Oregano and Cumin\n- Garlic Powder and Lemon Juice'

In [123]:
# test
graph_data_update_prompt = f"""
Node labels: {kg_meta_node_labels}
Relationship labels: {kg_meta_edge_labels}
Instruction: Save the contents into the database
Contents: {preprocess_result}
"""
chain.run(graph_data_update_prompt)



> Entering new  chain...
Generated Cypher:
CREATE (base:Ingredient {name: "cooked quinoa and white rice"})
CREATE (vegetable1:Ingredient {name: "diced red bell peppers"})
CREATE (vegetable2:Ingredient {name: "carrots"})
CREATE (vegetable3:Ingredient {name: "onions"})
CREATE (protein:Ingredient {name: "diced chicken"})
CREATE (seasoning1:Seasoning {name: "oregano"})
CREATE (seasoning2:Seasoning {name: "cumin"})
CREATE (seasoning3:Seasoning {name: "garlic powder"})
CREATE (lemon:Ingredient {name: "fresh lemon juice"})

CREATE (base)-[:MADE_FROM]->(recipe:Recipe)
CREATE (recipe)-[:HAS_PREPARATION]->(preparation:Preparation)
CREATE (preparation)-[:REQUIRES_SEASONING]->(seasoning1)
CREATE (preparation)-[:REQUIRES_SEASONING]->(seasoning2)
CREATE (preparation)-[:REQUIRES_SEASONING]->(seasoning3)
CREATE (preparation)-[:SERVED_WITH]->(lemon)
CREATE (recipe)-[:MADE_FROM]->(vegetable1)
CREATE (recipe)-[:MADE_FROM]->(vegetable2)
CREATE (recipe)-[:MADE_FROM]->(vegetable3)
CREATE (recipe)-[:MADE_F

'To save the contents into the database, you would need to store the following information:\n\n- Base of the dish: A mix of cooked quinoa and white rice.\n- Vegetables used: Diced red bell peppers, carrots, and onions.\n- Proteins used: Diced chicken.\n- Seasoning: A blend of herbs and spices including oregano, cumin, and garlic powder.\n- Finishing touch: A squeeze of fresh lemon juice.\n\nBy saving these details, you can easily retrieve and reference them when needed.'